#Cluster Config
1. Add the following to Cluster Configuration | Aadvanced Options | Spark |
  * Spark Config:  
   `spark.kryoserializer.buffer.max 1000M`  
   `spark.serializer org.apache.spark.serializer.KryoSerializer`  
  * Environment Variables:  
   `secret="<secret>"`  
   `SPARK_NLP_LICENSE="<SPARK_NLP_LICENSE>"`  
   `AWS_ACCESS_KEY_ID="<AWS_ACCESS_KEY_ID>"`  
   `AWS_SECRET_ACCESS_KEY="<AWS_SECRET_ACCESS_KEY>"`  
   `JSL_OCR_LICENSE="<SPARK OCR SECRET_KEY>"`  
   `PIP_EXTRA_INDEX_URL="https://pypi.johnsnowlabs.com/${secret}"`  
1. Restart cluster then run this notebook to install libraries  

####notes:
* https://nlp.johnsnowlabs.com/docs/en/install

In [2]:
%run ./Include

In [3]:
from os import getenv
secret = getenv('secret')
downloadFile(workingDir, "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-2.5.0-rc1.jar")

In [4]:
import json

def printResponse(response):
    print("status_code = " + str(response.status_code))
    print(json.dumps(response.json(), indent=2))

API_URL = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None) + "/api/2.0"
API_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
CLUSTER_ID = dbutils.notebook.entry_point.getDbutils().notebook().getContext().clusterId().getOrElse(None)
HEADERS={'Authorization': "Bearer " + API_TOKEN}

In [5]:
LIBRARIES={
  "cluster_id": CLUSTER_ID,
  "libraries": [
    {"maven": {"coordinates": "JohnSnowLabs:spark-nlp:2.4.5"}},
    {"pypi": {"package": "spark-nlp==2.5.0"}},
    {"pypi": {"package": "spark-nlp-jsl==2.5.0-rc1"}},
    {"jar": "dbfs:"+workingDir+"/spark-nlp-jsl-2.5.0-rc1.jar"}
  ]
}

response = requests.post(
  '%s/libraries/install' % (API_URL),
  headers=HEADERS,
  json=LIBRARIES
)

printResponse(response)

In [6]:
response = requests.get(
  '%s/libraries/cluster-status?cluster_id=%s' % (API_URL, CLUSTER_ID),
  headers=HEADERS
)
 
printResponse(response)